In [1]:

import os
import dlib
import time
import cv2 as cv
import numpy as np
from mtcnn.mtcnn import MTCNN
import face_recognition as facer
from tqdm import tnrange


In [2]:
pNum = "T000001"

proj = ".."
pimg = f"{proj}/Data/images"
pmdl = f"{proj}/Model"
pout = f"{proj}/Out/{pNum}"


In [3]:
modelv = "_81"
detector = dlib.get_frontal_face_detector()
%time predictor = dlib.shape_predictor(f"{pmdl}/shape_predictor{modelv}_face_landmarks.dat")


CPU times: user 117 ms, sys: 11.9 ms, total: 129 ms
Wall time: 132 ms


In [4]:
#
st, el = 1, 712


In [5]:

def f_xy(images):
    min_y, max_y, min_x, max_x = 112, 0, 112, 0
    cv_face = detector(cv.cvtColor(images, cv.COLOR_BGR2GRAY), 1)
    if cv_face:
        for face in cv_face:
            shape = predictor(images, face)
            for pt in shape.parts():
                min_y, max_y = min(pt.y, min_y), max(pt.y, max_y)
                min_x, max_x = min(pt.x, min_x), max(pt.x, max_x)
    else:
        min_y, max_y, min_x, max_x = 0, 112, 0, 112
    
    min_y, max_y, min_x, max_x = max(min_y,0), min(max_y,112), max(min_x,0), min(max_x,112)
    return min_y, max_y, min_x, max_x

print(
    f_xy(cv.imread(f"{pimg}/00001.jpg")), 
    f_xy(cv.imread(f"{pimg}/00003.jpg"))
)
%time f_xy(cv.imread(f"{pimg}/00001.jpg"))


(16, 112, 19, 102) (0, 112, 0, 112)
CPU times: user 7.32 ms, sys: 296 µs, total: 7.61 ms
Wall time: 7.37 ms


(16, 112, 19, 102)

In [6]:
%time dmtcnn = MTCNN()

def f_xy_mtcnn(images):
    rdetector = dmtcnn.detect_faces(images)
    min_y, max_y, min_x, max_x = 112, 0, 112, 0
    if rdetector:
        x, y, w, h = rdetector[0]["box"]
        min_y, max_y, min_x, max_x = y, y+h, x, x+w
    else:
        min_y, max_y, min_x, max_x = 0, 112, 0, 112
        
    min_y, max_y, min_x, max_x = max(min_y,0), min(max_y,112), max(min_x,0), min(max_x,112)
    return min_y, max_y, min_x, max_x

print(
    f_xy_mtcnn(cv.imread(f"{pimg}/00001.jpg")), 
    f_xy_mtcnn(cv.imread(f"{pimg}/00003.jpg")),
    f_xy_mtcnn(cv.imread(f"{pimg}/00489.jpg"))
)
%time f_xy_mtcnn(cv.imread(f"{pimg}/00001.jpg"))


CPU times: user 169 ms, sys: 13.6 ms, total: 182 ms
Wall time: 190 ms
(17, 112, 14, 99) (14, 111, 2, 80) (0, 112, 0, 112)
CPU times: user 205 ms, sys: 7.58 ms, total: 213 ms
Wall time: 200 ms


(17, 112, 14, 99)

In [7]:

def f_xy_facer(images):
    result = facer.face_landmarks(images)
    min_y, max_y, min_x, max_x = 112, 0, 112, 0
    if result:
        resultl = []
        for ir in result[0].values():
            resultl += ir
        for ir in resultl:
            min_y, max_y = min(ir[1], min_y), max(ir[1], max_y)
            min_x, max_x = min(ir[0], min_x), max(ir[0], max_x)
    else:
        min_y, max_y, min_x, max_x = 0, 112, 0, 112
    
    min_y, max_y, min_x, max_x = max(min_y,0), min(max_y,112), max(min_x,0), min(max_x,112)
    return min_y, max_y, min_x, max_x

print(
    f_xy_facer(cv.imread(f"{pimg}/00001.jpg")), 
    f_xy_facer(cv.imread(f"{pimg}/00003.jpg")),
    f_xy_facer(cv.imread(f"{pimg}/00489.jpg"))
)
%time f_xy_facer(cv.imread(f"{pimg}/00004.jpg"))


(36, 112, 19, 102) (0, 112, 0, 112) (0, 112, 0, 112)
CPU times: user 8.78 ms, sys: 281 µs, total: 9.07 ms
Wall time: 9.1 ms


(38, 112, 0, 86)

In [8]:
os.system(f"rm -rf {pout}/images/*")

0

In [9]:

#
def f_lf1(img1, img2):
    img1, img2 = cv.imread(img1), cv.imread(img2)
    return np.sqrt(np.sum(np.square(img1 - img2)))

print("SAME: %9.4f" % f_lf1(f"{pimg}/00001.jpg", f"{pimg}/00001.jpg"))
print("DIFF: %9.4f" % f_lf1(f"{pimg}/00001.jpg", f"{pimg}/00002.jpg"))
print("DIFF: %9.4f" % f_lf1(f"{pimg}/00001.jpg", f"{pimg}/00406.jpg"))

%time f_lf1(f"{pimg}/00001.jpg", f"{pimg}/00002.jpg")


SAME:    0.0000
DIFF: 1974.1583
DIFF: 1993.6366
CPU times: user 480 µs, sys: 136 µs, total: 616 µs
Wall time: 496 µs


1974.1583016566833

In [10]:

#
def _score(fimg):
    _img = cv.imread(fimg)
    result1 = facer.face_landmarks(_img)
    result2 = detector(cv.cvtColor(_img, cv.COLOR_BGR2GRAY), 1)
    
    if result1 or result2:
        return False
    else:
        return True

%time print(_score(f"{pimg}/00001.jpg"))
%time print(_score(f"{pimg}/00002.jpg"))


False
CPU times: user 13.4 ms, sys: 602 µs, total: 14 ms
Wall time: 13.6 ms
False
CPU times: user 14.5 ms, sys: 480 µs, total: 15 ms
Wall time: 14.9 ms


In [11]:
#
badl = []
for i in tnrange(st, el+1):
    pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
    if _score(pi):
        badl.append(i)
len(badl)


16

In [12]:

#
savl = []
for i in tnrange(st, el+1):
    pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
    op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    
    if not os.path.exists(op):
        img = cv.imread(pi)
        cv.imwrite(op, img)
        savl.append(1)
len(savl)


712